In [1]:
import json
import numpy as np
import re
import warnings
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import concurrent
import contractions
import en_core_web_sm
import logging as log
import multiprocessing
import pickle
import spacy
import sys
import os
import re
import threading
import time
from collections import Counter
from nltk.corpus import stopwords
from nltk import PorterStemmer
from spacy import displacy
from Stemmer import Stemmer
from word2number import w2n
import pickle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import re
from word2number import w2n
import os

In [23]:
pickfile = open('D:\College\Study\IRE\Project\data\pretrainedEmbeddings\embeddings.pickle','rb')
embeddings = pickle.load(pickfile)

In [24]:
def get_info(id, info_name, data_json):
    return data_json[id]['info'][0][info_name]

In [25]:
def load_data(file_path):

    with open(file_path, 'r', encoding="utf-8") as file:
        tweets = []
        target_num = []
        offset = []
        target_cashtag = []
        relation = []

        data_json = json.load(file)

        def getAspects(aspect):
            aspect = aspect.replace('[', '')
            aspect = aspect.replace(']', '')
            aspect = aspect.replace('\'', '')
            return aspect.split('/')

        for id in data_json:
            tweets.append(id['tweet'].lower())
            target_num.append(id['target_num'])
            offset.append(id['offset'])
            target_cashtag.append(id['target_cashtag'].lower())
            relation.append(id['relation'])
            

    return tweets, target_num, offset, target_cashtag, relation


In [26]:
def __removePunctuations(sequence,ner_tags):
    #########################################################################################
    # This method removes any punctuations and gets only the text from the given sequence.
    #########################################################################################
    try:
        if sequence is not None and sequence.strip() != "":
            if sequence in ner_tags:
                return re.sub('[^A-Za-z0-9%$.]+',' ',sequence)
            else:
                return re.sub('[^A-Za-z0-9$%]+',' ',sequence)
        return sequence # return sequence as is without any changes
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        err = "Error occurred while removing punctuations in the sequence '{0}'. Error is: {1}; {2}".format(sequence, str(exc_type), str(exc_value))
        raise Exception(err)


In [27]:
def __clean_data(data,ner_tags):
    #########################################################################################
    # This method cleans the data by applying following API's and returns list of preprocessed tokens
    #     data.lower()
    #     __expandContractions(data)
    #     __removeHtmlTags(data)
    #     __replaceurls(data)
    #     __removePunctuations(data)
    #     data.split(" ")
    #########################################################################################
    #data = __expandContractions(data)
    #data = __removeHtmlTags(data) # remove tags
    #data = __replaceurls(data)
    data = data.replace("$","$ ")
    data = data.replace("%"," % ")
    data = data.split(" ")
    clean = []
    for word in data:
        w_clean = __removePunctuations(word,ner_tags)
        clean.extend(w_clean.split(" "))
    data = clean
    data = [word if word.isupper() and word.lower() in ner_tags else word.lower() for word in data]
    dollar_list = ['$','k','%']
    data = [d for d in data if len(d)>=2 or d in dollar_list]
#     clean_list.append(data)
    return data


In [28]:
def get_embedded_tweets(cleaned_tweets):
    vec = np.zeros([len(cleaned_tweets),300], dtype = 'float32') 
    c=0
    for i in cleaned_tweets:
        for j in i:
            try:
                j=str(j)
                k=embeddings[j]
                vec[c]=(vec[c]+np.array(k))
            except:
                continue
        c=c+1
    return vec

In [29]:
def __applyner(sequence):
    #########################################################################################
    # This method applies NER and returns the sequence according to the operation performed based on NER tag.
    #########################################################################################
    pickfile = open('tickermapping.pickle','rb')
    tickermapping = pickle.load(pickfile)
    ner_tags = []
    doc = nlp(sequence)  # applying NER
    for X in doc.ents:
        # If the NER class is ORG
        if X.label_ == 'ORG':
            "X.text can take microsoft corp or abcd name MSFT"
            text = X.text
            if text in tickermapping.keys():
                text = tickermapping[X.text]
            text = re.sub(r'[^\w\s]', '', X.text).lower()
            if 'inc' in text:
                text = text.replace('inc', '')
            if 'ltd' in text:
                text = text.replace('ltd', '')
            if 'llp' in text:
                text = text.replace('llp', '')
            if 'limited' in text:
                text = text.replace('limited', '')
            if 'corp' in text:
                text = text.replace('corp', '')
            if 'the' in text.lower():
                text = text.replace('the','')
            sequence = sequence.replace(X.text, text)
            ner_tags.extend(text.lower().split(" "))
        # If NER class is MONEY
        if X.label_ == 'MONEY':
            new_X = X.text.lower()
            if 'approximately' in new_X:  # Remove all the words which might appear in NER money class
                new_X = new_X.replace('approximately', '')
            if 'per' in new_X:
                new_X = new_X.replace('per', '')
            if 'to' in new_X:
                new_X = new_X.replace('to', '')
            if 'and' in new_X:
                new_X = new_X.replace('and', '')
            if 'between' in new_X:
                new_X = new_X.replace('between', '')
            if 'phone' in new_X:
                continue
            # Apply NER for the string which is obtained after removing other words this gives $200, $500 as separate ones
            if '$' not in new_X:
                new_X = "$"+new_X
            doc1 = nlp(new_X)
            for Y in doc1.ents:
                money = Y.text
                if ' ' not in money:
                    act_money = money.replace(',', '')  # Actual Money
                    #act_money = act_money.replace('.','')
                    sequence = sequence.replace(Y.text, act_money)  # Replace original money text with actual money
                    ner_tags.append(act_money)
                    # print(act_money)
                else:
                    money = Y.text[Y.text.find("$") + 1:]
                    k = money.find(' ')
                    try:
                        act_money = float(money[:k].replace(',', ''))
                        #act_money = act_money.replace('.','')
                        money_conv = w2n.word_to_num(money[k:])  # Conversion of word types million to *1e6
                        sequence = sequence.replace(Y.text, "$ "+str(act_money * money_conv))  # Replace original money text with actual money
                        #print("Converted from", money, act_money * money_conv)
                    except:
                        continue  # if any exception dont modify the original sentence and continue
        # If NER class is LAW
        if X.label_ == 'LAW':
            new_X = X.text
            new_X = re.sub(r'[\d.!?\-"]', '', new_X)
            if 'the' in new_X.lower():
                new_X = new_X.lower().replace('the', '')
            if 'of' in new_X.lower():
                new_X = new_X.lower().replace('of', '')
            if 'section' in new_X.lower():
                new_X = new_X.lower().replace('section', '')
            sequence = sequence.replace(X.text, new_X)
            ner_tags.extend(new_X.split(" "))
        # If NER class is Location
        if X.label_ == 'GPE':
            new_X = X.text.lower()
            new_X = re.sub(r'[\d.!?\-"]', '', new_X)
            if 'the' in new_X.lower():
                new_X = new_X.lower().replace('the', '')
            if '.' in new_X.lower():
                new_X = new_X.lower().replace('.', '')
            sequence = sequence.replace(X.text, new_X)
            ner_tags.extend(new_X.split(" "))
        # If NER class is Person
        if X.label_ == 'PERSON':
            new_X = X.text.lower()
            new_X = re.sub(r'[\d.!?\-"]', '', new_X)
            if 'the' in new_X.lower():
                new_X = new_X.lower().replace('the', '')
            if '.' in new_X.lower():
                new_X = new_X.lower().replace('.', '')
            sequence = sequence.replace(X.text, new_X)
            ner_tags.extend(new_X.split(" "))
        if X.label_ == 'CARDINAL':
            number = X.text
            number = number.replace(',','')
            #number = number.replace('.','')
            if number.isnumeric():
                sequence = sequence.replace(X.text, number)
        if X.label_ == 'QUANTITY':
            quantity = X.text.split(" ")
            for number in quantity:
                number = number.replace(',','')
                number = number.replace('.','')
                if number.isnumeric():
                    sequence = sequence.replace(X.text, number)
        if X.label_ == "PERCENT":
            percent = X.text.replace('%','')
            ner_tags.append(percent)
    return sequence,ner_tags


In [30]:
def preprocess(sentences):
    cleaned_sentences = []
    for line in sentences:
        line = line.replace("$.","$0.")
        # Apply NER for the line
        line,ner_tags = __applyner(line)
        tokens = __clean_data(line,ner_tags)
        cleaned_sentences.append(tokens)
    return cleaned_sentences

In [31]:
tweets, target_num, offset, target_cashtag, y_train = load_data(r"D:\College\Study\IRE\Project\data\Extrinsic Task 2 (Binary Classification) Data\FinNum_training_v3.json")
cleaned_tweets_train = preprocess(tweets)

In [32]:
tweets[0]

'$xxii scott gottlieb, commissioner of fda speech transcript from november 3rd, less than 2 months left in year then.'

In [33]:
tweets, target_num, offset, target_cashtag, y_test = load_data(r"D:\College\Study\IRE\Project\data\Extrinsic Task 2 (Binary Classification) Data\FinNum_dev_v3.json")
cleaned_tweets_test = preprocess(tweets)

In [34]:
X_train = get_embedded_tweets(cleaned_tweets_train)
X_test = get_embedded_tweets(cleaned_tweets_test)

In [35]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [36]:
clf=svm.SVC(C=0.3)
clf.fit(X_train,y_train)
prediction=clf.predict(X_test)

pred_labels=prediction==y_test
acc=0.0
for i in pred_labels:
    if i==True:
        acc+=1
        
print ('SVM accuracy=',(acc)/len(pred_labels)*100,'%')
print(classification_report(y_test,prediction,labels=np.unique(y_test)))
print("\n\nConfusion_Matrix")
print(confusion_matrix(y_test,prediction))
print("F1: ",f1_score(y_test,prediction))

SVM accuracy= 83.23754789272031 %
              precision    recall  f1-score   support

           0       1.00      0.10      0.18       194
           1       0.83      1.00      0.91       850

    accuracy                           0.83      1044
   macro avg       0.91      0.55      0.54      1044
weighted avg       0.86      0.83      0.77      1044



Confusion_Matrix
[[ 19 175]
 [  0 850]]
F1:  0.9066666666666667


In [37]:
from sklearn.ensemble import RandomForestClassifier
rlf = RandomForestClassifier()
rlf.fit(X_train,y_train)
prediction=rlf.predict(X_test)
pred_labels=prediction==y_test
acc=0.0
for i in pred_labels:
    if i==True:
        acc+=1
print ('Random forest accuracy=',(acc)/len(pred_labels)*100,'%')

from sklearn.metrics import f1_score
# print(classification_report(y_test,prediction,labels=np.unique(y_test)))
# print("\n\nConfusion_Matrix")
# print(confusion_matrix(y_test,prediction))
print("F1 score: ",f1_score(y_test,prediction))

Random forest accuracy= 85.727969348659 %
F1 score:  0.9189777052746058


In [38]:
from sklearn.ensemble import GradientBoostingClassifier
cl=GradientBoostingClassifier()
cl.fit(X_train,y_train)
prediction=cl.predict(X_test)
pred_labels=prediction==y_test
acc=0.0
for i in pred_labels:
    if i==True:
        acc+=1
print ('SVM accuracy=',(acc)/len(pred_labels)*100,'%')

from sklearn.metrics import classification_report
print(classification_report(y_test,prediction,labels=np.unique(y_test)))

from sklearn.metrics import confusion_matrix
print("\n\nConfusion_Matrix")
print(confusion_matrix(y_test,prediction))
from sklearn.metrics import f1_score
print("F1: ",f1_score(y_test,prediction))

SVM accuracy= 86.30268199233716 %
              precision    recall  f1-score   support

           0       0.98      0.27      0.42       194
           1       0.86      1.00      0.92       850

    accuracy                           0.86      1044
   macro avg       0.92      0.63      0.67      1044
weighted avg       0.88      0.86      0.83      1044



Confusion_Matrix
[[ 52 142]
 [  1 849]]
F1:  0.9223248234655079


In [43]:
#pickle.dump(clf, open('model_pretrained_classification_svc.pickle', 'wb'))
#pickle.dump(clf, open('model_our_classification_svc.pickle', 'wb'))

In [44]:
#pickle.dump(rlf, open('model_pretrained_classification_rforest.pickle', 'wb'))
#pickle.dump(rlf, open('model_our_classification_rforest.pickle', 'wb'))

In [45]:
#pickle.dump(cl, open('model_pretrained_classification_gboost.pickle', 'wb'))
#pickle.dump(cl, open('model_our_classification_gboost.pickle', 'wb'))